In [1]:
import mmh3

In [2]:
import functools

In [3]:
import iteration_utilities

In [4]:
import random

In [5]:
import numpy as np

In [6]:
import math

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.python import debug as tf_debug

In [9]:
from enum import IntEnum

In [10]:
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


In [11]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
training_data, testing_data = tf.keras.datasets.fashion_mnist.load_data()

In [12]:
number_of_classes = int(np.max(testing_data[1])) + 1
number_of_input_channels = 1

In [13]:
number_of_experts = 1

In [14]:
number_of_hidden_layers = 20

In [15]:
number_of_features_per_layer = 32

In [16]:
paddings = (0, 0), (2, 2), (2, 2)

In [17]:
random_seed = number_of_hidden_layers

In [18]:
def offset_initializer():
    return tf.random_normal_initializer(stddev=1.0)
#     return tf.random_uniform_initializer(minval=-3.0, maxval=3.0)

In [19]:
def grid_sample(images, offset_x, offset_y):
    tf.assert_rank(images, 4)
    tf.assert_rank(offset_x, 4)
    tf.assert_rank(offset_y, 4)
    image_shape = tf.shape(images)
    current_batch_size = image_shape[0]
    width = image_shape[1]
    height = image_shape[2]
    number_of_features = image_shape[3]
    
    def check_width(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, width - 1), 0)
    
    def check_height(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, height - 1), 0)
    
    offset_left = tf.floor(offset_x)
    offset_top = tf.floor(offset_y)
    tf.assert_equal(tf.shape(offset_left), image_shape)
    tf.assert_equal(tf.shape(offset_top), image_shape)

    
    eps = 1e-7

    factor_right = offset_x - offset_left + eps
    factor_left = 1.0 + 2.0 * eps - factor_right
    factor_bottom = offset_y - offset_top + eps
    factor_top = 1.0 + 2.0 * eps - factor_bottom
    
    image_index, x_index, y_index, feature_index = tf.meshgrid(
        tf.range(current_batch_size, dtype=tf.int32),
        tf.range(width, dtype=tf.int32),
        tf.range(height, dtype=tf.int32),
        tf.range(number_of_features, dtype=tf.int32),
        indexing='ij',
    )

    left_index = check_width(x_index + tf.cast(offset_left, tf.int32))
    top_index = check_height(y_index + tf.cast(offset_top, tf.int32))

    right_index = left_index + 1
    bottom_index = top_index + 1
    
    tf.assert_rank(left_index, 4)
    tf.assert_rank(right_index, 4)
    tf.assert_rank(bottom_index, 4)
    tf.assert_rank(top_index, 4)
    
    images_top_left = tf.gather_nd(images, tf.stack((image_index, left_index, top_index, feature_index), axis=4))
    images_top_right = tf.gather_nd(images, tf.stack((image_index, right_index, top_index, feature_index), axis=4))
    images_bottom_left = tf.gather_nd(images, tf.stack((image_index, left_index, bottom_index, feature_index), axis=4))
    images_bottom_right = tf.gather_nd(images, tf.stack((image_index, right_index, bottom_index, feature_index), axis=4))

    tf.assert_rank(images_top_left, 4)
    tf.assert_rank(images_top_right, 4)
    tf.assert_rank(images_bottom_left, 4)
    tf.assert_rank(images_bottom_right, 4)

    lerp_top = factor_left * images_top_left + factor_right * images_top_right
    lerp_bottom = factor_left * images_bottom_left + factor_right * images_bottom_right
    output = factor_top * lerp_top + factor_bottom * lerp_bottom
    tf.assert_rank(output, 4)
    return output

In [20]:
def offnet(images, number_of_features, number_of_experts=None, name=None):
    with tf.variable_scope(name, default_name="offnet"):
        input_shape = images.shape
        if number_of_experts is None:
            number_of_experts = int(input_shape[0])
        number_of_input_features = int(input_shape[4])
        score_weight = tf.get_variable(
            name="score_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_x_weight = tf.get_variable(
            name="offset_x_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_y_weight = tf.get_variable(
            name="offset_y_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_x_bias = tf.get_variable(
            name="offset_x_bias",
            initializer=offset_initializer(),
            dtype=tf.float32,
            shape=(number_of_experts, 1, number_of_features),
        )
        offset_y_bias = tf.get_variable(
            name="offset_y_bias",
            initializer=offset_initializer(),
            dtype=tf.float32,
            shape=(number_of_experts, 1,  number_of_features),
        )
#         def scale_initializer():
#             return tf.sqrt(weight_scale / number_of_input_features)
#         score_scale = tf.get_variable(
#             name="score_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
#         offset_x_scale = tf.get_variable(
#             name="offset_x_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
#         offset_y_scale = tf.get_variable(
#             name="offset_y_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
        
        dynamic_image_shape = tf.shape(images)
        images_3d = tf.reshape(
            images,
            (
                number_of_experts,
                dynamic_image_shape[1] * dynamic_image_shape[2] * dynamic_image_shape[3],
                number_of_input_features
            )
        )
        def images_xw_plus_b(w, b):
            tf.matmul(images_3d, w) + b
        
        def to_4d(images):
            tf.assert_equal(tf.shape(images), (number_of_experts, dynamic_image_shape[1] * dynamic_image_shape[2] * dynamic_image_shape[3], number_of_features))
            return tf.reshape(
                images,
                (
                    number_of_experts * dynamic_image_shape[1],
                    dynamic_image_shape[2],
                    dynamic_image_shape[3],
                    number_of_features
                )
            )
        return tf.reshape(
            grid_sample(
                to_4d(tf.matmul(images_3d, score_weight)),
                to_4d(tf.matmul(images_3d, offset_x_weight) + offset_x_bias),
                to_4d(tf.matmul(images_3d, offset_y_weight) + offset_y_bias)
            ),
            (
                number_of_experts,
                dynamic_image_shape[1],
                dynamic_image_shape[2],
                dynamic_image_shape[3],
                number_of_features
            )
        )        

In [21]:
number_of_horizontal_samples = 3
number_of_vertical_samples = 3

In [22]:
def model_fn(features, labels, mode, params, config):
    random.seed(number_of_hidden_layers)
    padded_input = tf.expand_dims(
        tf.pad(tf.cast(features, tf.float32) / 255.0 - 0.5, paddings),
        axis=3
    )
    width = int(padded_input.shape[1])
    height = int(padded_input.shape[2])
    
    tf.assert_equal(width, 32)
    tf.assert_equal(height, 32)
    
    layers = functools.reduce(
        lambda features, i: tf.nn.relu(tf.layers.batch_normalization(
            tf.layers.conv2d(features, number_of_features_per_layer, (3, 3), padding='same'),
            training=mode is tf.estimator.ModeKeys.TRAIN
        )),
        range(number_of_hidden_layers),
        padded_input
    )
    score_layer = tf.layers.conv2d(layers, number_of_classes, (3, 3), padding='same')
    scores = tf.reduce_mean(
        score_layer[
            :,
            int(width / (number_of_horizontal_samples + 1)):int(width * number_of_horizontal_samples / (number_of_horizontal_samples + 1)):int(width / (number_of_horizontal_samples + 1)),
            int(height / (number_of_horizontal_samples + 1)):int(height * number_of_horizontal_samples / (number_of_horizontal_samples + 1)):int(height / (number_of_horizontal_samples + 1)),
            :
        ],
        axis=(1, 2)
    )

    probabilities = tf.nn.softmax(logits=scores)
    predicted_classes = tf.argmax(scores, 1)
    predictions = {
        'probabilities' : probabilities,
        'scores': scores,
        'class': predicted_classes,
    }

    optimizer = tf.train.AdamOptimizer(epsilon=0.1)
    if labels is None:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
        )
    else:
        loss = tf.losses.softmax_cross_entropy(logits=scores, onehot_labels=tf.one_hot(labels, number_of_classes))
        eval_metric_ops = {
            'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)
        }
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            return tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=predictions,
                loss=loss,
                train_op=optimizer.minimize(loss, global_step=tf.train.get_global_step()),
                eval_metric_ops=eval_metric_ops,
            )

In [23]:
run_config = tf.estimator.RunConfig(
    model_dir=f"models/cnnreluoffnet{number_of_features_per_layer}x{number_of_hidden_layers}",
    session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(
            allow_growth=True,
        ),
        graph_options=tf.GraphOptions(
            optimizer_options=tf.OptimizerOptions(
                global_jit_level=tf.OptimizerOptions.ON_2,
                do_function_inlining=True,
                do_constant_folding=True,
                do_common_subexpression_elimination=True,
            ),
        )
    )
)

In [24]:
estimator = tf.estimator.Estimator(model_fn, config=run_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/cnnreluoffnet32x20', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
graph_options {
  optimizer_options {
    do_common_subexpression_elimination: true
    do_constant_folding: true
    do_function_inlining: true
    global_jit_level: ON_2
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f115a27a390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
batch_size = 32

In [26]:
def training_dataset():
    return tf.data.Dataset.from_tensor_slices(training_data).shuffle(1000).batch(batch_size)

In [27]:
def testing_dataset():
    return tf.data.Dataset.from_tensor_slices(testing_data).batch(batch_size)

In [28]:
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
# estimator.train(training_dataset,hooks=[hook])

In [29]:
tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(training_dataset),
    eval_spec=tf.estimator.EvalSpec(testing_dataset, throttle_secs=1800)
)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 1800 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/cnnreluoffnet32x20/model.ckpt.
INFO:tensorflow:loss = 2.52204, step = 0
INFO:tensorflow:global_step/sec: 48.0143
INFO:tensorflow:loss = 2.2300587, step = 100 (2.085 sec)
INFO:tensorflow:global_step/sec: 67.0281
INFO:tensorflow:loss = 1.5156474, step = 200 (1.492 sec)
INFO:tensorflow:global_step/sec: 68.5234
INFO:tensorflow:loss = 1.2275314, step = 300 (1.459 sec)
INFO:tensorflow:global_step/sec: 67.3661
INFO:tensorflow:loss = 0.9209537, step = 400 (1.485 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:loss = 0.1920131, step = 4950 (1.607 sec)
INFO:tensorflow:global_step/sec: 61.2374
INFO:tensorflow:loss = 0.5973839, step = 5050 (1.634 sec)
INFO:tensorflow:global_step/sec: 62.2279
INFO:tensorflow:loss = 0.29308423, step = 5150 (1.606 sec)
INFO:tensorflow:global_step/sec: 63.1633
INFO:tensorflow:loss = 0.24327496, step = 5250 (1.584 sec)
INFO:tensorflow:global_step/sec: 60.9355
INFO:tensorflow:loss = 0.08477466, step = 5350 (1.641 sec)
INFO:tensorflow:global_step/sec: 61.0059
INFO:tensorflow:loss = 0.41891694, step = 5450 (1.639 sec)
INFO:tensorflow:global_step/sec: 61.7197
INFO:tensorflow:loss = 0.32252464, step = 5550 (1.620 sec)
INFO:tensorflow:Saving checkpoints for 5625 into models/cnnreluoffnet32x20/model.ckpt.
INFO:tensorflow:Loss for final step: 0.2793618.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-19-01:30:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters fr

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/cnnreluoffnet32x20/model.ckpt-9375
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9376 into models/cnnreluoffnet32x20/model.ckpt.
INFO:tensorflow:loss = 0.33435103, step = 9375
INFO:tensorflow:global_step/sec: 49.8441
INFO:tensorflow:loss = 0.37520385, step = 9475 (2.009 sec)
INFO:tensorflow:global_step/sec: 66.135
INFO:tensorflow:loss = 0.38782686, step = 9575 (1.511 sec)
INFO:tensorflow:global_step/sec: 66.4773
INFO:tensorflow:loss = 0.18953666, step = 9675 (1.504 sec)
INFO:tensorflow:global_step/sec: 64.8803
INFO:tensorflow:loss = 0.28045788, step = 9775 (1.542 sec)
INFO:tensorflow:global_step/sec: 63.6395
INFO:tensorflow:loss = 0.20175296, step = 9875 (1.571 sec)
INFO:tensorflow:global_step/sec: 62.6628
INFO:tensorflow:loss = 0.6208861, step = 9975 (1.596 sec)
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 61.9086
INFO:tensorflow:loss = 0.16155349, step = 14525 (1.616 sec)
INFO:tensorflow:global_step/sec: 62.5813
INFO:tensorflow:loss = 0.276615, step = 14625 (1.597 sec)
INFO:tensorflow:global_step/sec: 61.3572
INFO:tensorflow:loss = 0.37732506, step = 14725 (1.630 sec)
INFO:tensorflow:global_step/sec: 61.5008
INFO:tensorflow:loss = 0.21568033, step = 14825 (1.626 sec)
INFO:tensorflow:global_step/sec: 61.5123
INFO:tensorflow:loss = 0.37670654, step = 14925 (1.626 sec)
INFO:tensorflow:Saving checkpoints for 15000 into models/cnnreluoffnet32x20/model.ckpt.
INFO:tensorflow:Loss for final step: 0.093061596.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-19-01:34:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/cnnreluoffnet32x20/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evalua

INFO:tensorflow:Restoring parameters from models/cnnreluoffnet32x20/model.ckpt-18750
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 18751 into models/cnnreluoffnet32x20/model.ckpt.
INFO:tensorflow:loss = 0.2638559, step = 18750
INFO:tensorflow:global_step/sec: 49.6832
INFO:tensorflow:loss = 0.20081174, step = 18850 (2.015 sec)
INFO:tensorflow:global_step/sec: 67.141
INFO:tensorflow:loss = 0.32972488, step = 18950 (1.489 sec)
INFO:tensorflow:global_step/sec: 65.5329
INFO:tensorflow:loss = 0.06606376, step = 19050 (1.526 sec)
INFO:tensorflow:global_step/sec: 65.7898
INFO:tensorflow:loss = 0.1626797, step = 19150 (1.520 sec)
INFO:tensorflow:global_step/sec: 66.8143
INFO:tensorflow:loss = 0.17542283, step = 19250 (1.495 sec)
INFO:tensorflow:global_step/sec: 67.6251
INFO:tensorflow:loss = 0.32583845, step = 19350 (1.480 sec)
INFO:tensorflow:global_step/sec: 64.74
INFO:tensorflow:loss = 0.47778034, step = 19450 (1.545

...

INFO:tensorflow:loss = 1.971576e-05, step = 351725 (1.596 sec)
INFO:tensorflow:global_step/sec: 62.6265
INFO:tensorflow:loss = 1.0132203e-05, step = 351825 (1.596 sec)
INFO:tensorflow:global_step/sec: 62.6427
INFO:tensorflow:loss = 1.6639218e-05, step = 351925 (1.598 sec)
INFO:tensorflow:global_step/sec: 61.6071
INFO:tensorflow:loss = 3.3996916e-05, step = 352025 (1.623 sec)
INFO:tensorflow:global_step/sec: 62.2125
INFO:tensorflow:loss = 1.6501865e-05, step = 352125 (1.607 sec)
INFO:tensorflow:global_step/sec: 61.757
INFO:tensorflow:loss = 1.6892893e-05, step = 352225 (1.619 sec)
INFO:tensorflow:global_step/sec: 61.504
INFO:tensorflow:loss = 4.3771197e-06, step = 352325 (1.626 sec)
INFO:tensorflow:global_step/sec: 61.6377
INFO:tensorflow:loss = 7.680098e-05, step = 352425 (1.622 sec)
INFO:tensorflow:Saving checkpoints for 352500 into models/cnnreluoffnet32x20/model.ckpt.
INFO:tensorflow:Loss for final step: 1.2095196e-05.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling m

INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-06-19-04:04:01
INFO:tensorflow:Saving dict for global step 356250: accuracy = 0.916875, global_step = 356250, loss = 0.5921476
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/cnnreluoffnet32x20/model.ckpt-356250
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 356251 into models/cnnreluoffnet32x20/model.ckpt.
INFO:tensorflow:loss = 1.1569048e-05, step = 356250
INFO:tensorflow:global_step/sec: 49.5614
INFO:tensorflow:loss = 3.472352e-05, step = 356350 (2.020 sec)
INFO:tensorflow:global_step/sec: 66.9212
INFO:tensorflow:loss = 2.1209325e-05, step = 356450 (1.494 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 63.3447
INFO:tensorflow:loss = 1.9846178e-05, step = 360800 (1.579 sec)
INFO:tensorflow:global_step/sec: 62.1707
INFO:tensorflow:loss = 1.2032576e-06, step = 360900 (1.609 sec)
INFO:tensorflow:global_step/sec: 64.781
INFO:tensorflow:loss = 2.972551e-05, step = 361000 (1.543 sec)
INFO:tensorflow:global_step/sec: 62.7151
INFO:tensorflow:loss = 3.5138364e-05, step = 361100 (1.595 sec)
INFO:tensorflow:global_step/sec: 61.0784
INFO:tensorflow:loss = 9.1525635e-06, step = 361200 (1.637 sec)
INFO:tensorflow:global_step/sec: 61.7373
INFO:tensorflow:loss = 1.6814858e-05, step = 361300 (1.619 sec)
INFO:tensorflow:global_step/sec: 62.2914
INFO:tensorflow:loss = 9.413332e-06, step = 361400 (1.605 sec)
INFO:tensorflow:global_step/sec: 63.5813
INFO:tensorflow:loss = 6.816993e-06, step = 361500 (1.574 sec)
INFO:tensorflow:global_step/sec: 61.7752
INFO:tensorflow:loss = 1.077276e-05, step = 361600 (1.618 sec)
INFO:tensorflow:global_step/sec: 61.1743
INFO:tensorflow:los

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-06-19-04:08:08
INFO:tensorflow:Saving dict for global step 365625: accuracy = 0.916875, global_step = 365625, loss = 0.5917137
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/cnnreluoffnet32x20/model.ckpt-365625
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 365626 into models/cnnr

KeyboardInterrupt: 

In [ ]:
# estimator.evaluate(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1))

In [ ]:
# tuple(estimator.predict(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1)))

In [ ]:
# estimator.train(input_fn=training_dataset, steps=1)